<a href="https://colab.research.google.com/github/WellingtonFidelis/tpm-rl-bc-big-data/blob/main/bc_arch_big_data_students_test_grades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting student`s test grades

## Pratic task 1

> Bootcamp: **Big Data Architect**
>
> Module 1

This job is to train skills data science about.

---
---

> ## Context
> Business problem

A higher education teacher proposed an experiment to develop a predictive model that would use the hours dedicated by students to their studies to anticipate the grades obtained in the final assessments of the subject. To do this, he compiled data on the time spent studying and the results of the exams of a group of students from previous semesters.

From this, he concluded that the most effective approach to solving this problem would be to create a linear regression Machine Learning algorithm.

In this way, the model would be able to learn from the historical data collected and predict the likely results based on a new student`s hours of study by a new student.

## Environments configuration

In [1]:
from platform import python_version
print('Language Python vesrion used in this notebook is: ', python_version())

Language Python vesrion used in this notebook is:  3.10.12


In [2]:
!cat /etc/*release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.3 LTS"
PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.3 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


Creating a folder to downloads.

In [3]:
!mkdir Downloads

mkdir: cannot create directory ‘Downloads’: File exists


In [4]:
!ls -la

total 20
drwxr-xr-x 1 root root 4096 Apr 25 23:06 .
drwxr-xr-x 1 root root 4096 Apr 25 23:05 ..
drwxr-xr-x 4 root root 4096 Apr 24 13:25 .config
drwxr-xr-x 3 root root 4096 Apr 25 23:06 Downloads
drwxr-xr-x 1 root root 4096 Apr 24 13:26 sample_data


## Loading libraries

In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from os import path

> ## Loading datasets
> Data **collection**

Verifying if the dataset exists.

In [6]:
dataset_study_hours_exist = path.exists("./Downloads/horas_estudo/horas_estudo.csv")
dataset_study_hours_exist

True

Downloading and extracting the dataset if it doesn't exist.

In [7]:
if not dataset_study_hours_exist:
  !wget https://leandrolessa.com.br/wp-content/uploads/2024/04/horas_estudo.zip -P ./Downloads

In [8]:
!ls -la Downloads/

total 16
drwxr-xr-x 3 root root 4096 Apr 25 23:06 .
drwxr-xr-x 1 root root 4096 Apr 25 23:06 ..
drwxr-xr-x 2 root root 4096 Apr 25 23:08 horas_estudo
-rw-r--r-- 1 root root  776 Apr 17 19:51 horas_estudo.zip


In [9]:
!unzip -o ./Downloads/horas_estudo.zip -d ./Downloads/horas_estudo/

Archive:  ./Downloads/horas_estudo.zip
  inflating: ./Downloads/horas_estudo/horas_estudo.csv  


In [10]:
!ls -la ./Downloads/horas_estudo/

total 12
drwxr-xr-x 2 root root 4096 Apr 25 23:09 .
drwxr-xr-x 3 root root 4096 Apr 25 23:06 ..
-rw-r--r-- 1 root root 1579 Apr 17 15:34 horas_estudo.csv


In [11]:
df_h_study = pd.read_csv(
    filepath_or_buffer="./Downloads/horas_estudo/horas_estudo.csv",
    sep=";",
    header="infer",
)
df_h_study.head(n=5)

,horas_estudo,nota_avaliacao
0,"16,07","40,05"
1,"20,41","66,49"
2,"34,29","91,5"
3,"34,1","87,09"
4,"32,34","69,3"


> ## Checking the dataset
> Data cleaning and preparation

In [12]:
df_h_study.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   horas_estudo    123 non-null    object
 1   nota_avaliacao  123 non-null    object
dtypes: object(2)
memory usage: 2.0+ KB


⚠ The variable ara floats number but are storage as string and the number is in Portuguese currency which makes calculations difficult

In [13]:
df_h_study.describe()

,horas_estudo,nota_avaliacao
count,123,123
unique,100,100
top,"12,71","48,54"
freq,2,2


In [14]:
df_h_study.shape

(123, 2)

In [15]:
df_h_study.duplicated().sum()

23

There is 23 observations duplicated, however looking for the business there are possibilities that the students can get score equal others.

The percentage of duplicated observations is:

In [16]:
print("{:.2f} %".format((df_h_study.shape[0] / df_h_study.duplicated().sum())))

5.35 %


In [17]:
df_h_study.horas_estudo.isna().sum()

0

In [18]:
df_h_study.horas_estudo.isnull().sum()

0

There are not NA or NULL in 'horas_estudo's variable.

In [19]:
df_h_study.nota_avaliacao.isna().sum()

0

In [20]:
df_h_study.nota_avaliacao.isnull().sum()

0

There are not NA or NULL in 'notas_avaliacao's variable.

> ## Variable analyses
> Exploratory data analysis (EDA)

In [21]:
df_h_study.columns

Index(['horas_estudo', 'nota_avaliacao'], dtype='object')

In [22]:
df_h_study = df_h_study.rename(columns={
    "horas_estudo": "HR_STUDY",
    "nota_avaliacao": "EVALUATION_SCORE"
})
df_h_study.columns

Index(['HR_STUDY', 'EVALUATION_SCORE'], dtype='object')

Changing the data types of the variables to calculate de correlattion.

In [23]:
df_h_study.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   HR_STUDY          123 non-null    object
 1   EVALUATION_SCORE  123 non-null    object
dtypes: object(2)
memory usage: 2.0+ KB


In [24]:
df_h_study.HR_STUDY = df_h_study.HR_STUDY.map(lambda x: str(x).replace(".", "").replace(",", "."))
df_h_study.HR_STUDY.head(n=5)

0    16.07
1    20.41
2    34.29
3     34.1
4    32.34
Name: HR_STUDY, dtype: object

In [25]:
df_h_study.EVALUATION_SCORE = df_h_study.EVALUATION_SCORE.map(lambda x: str(x).replace(".", "").replace(",", "."))
df_h_study.EVALUATION_SCORE.head(n=5)

0    40.05
1    66.49
2     91.5
3    87.09
4     69.3
Name: EVALUATION_SCORE, dtype: object

In [26]:
df_h_study.HR_STUDY.max()

'9.75'

In [27]:
df_h_study.EVALUATION_SCORE.max()

'99.22'

In [28]:
df_h_study.describe()

,HR_STUDY,EVALUATION_SCORE
count,123,123
unique,100,100
top,12.71,48.54
freq,2,2


- There is no score over 100, it is correct.
-

In [29]:
df_h_study.HR_STUDY = df_h_study.HR_STUDY.astype(dtype=np.float32)
df_h_study.EVALUATION_SCORE = df_h_study.EVALUATION_SCORE.astype(dtype=np.float64)

df_h_study.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   HR_STUDY          123 non-null    float32
 1   EVALUATION_SCORE  123 non-null    float64
dtypes: float32(1), float64(1)
memory usage: 1.6 KB


In [30]:
df_h_study.describe()

,HR_STUDY,EVALUATION_SCORE
count,123.000000,123.000000
mean,20.594635,64.059106
std,11.140682,22.533967
min,0.380000,10.300000
25%,12.250000,48.155000
50%,19.430000,62.320000
75%,30.330000,86.260000
max,39.160000,100.000000


In [31]:
df_corr = df_h_study.corr(method="pearson")
df_corr

,HR_STUDY,EVALUATION_SCORE
HR_STUDY,1.000000,0.705654
EVALUATION_SCORE,0.705654,1.000000


In [32]:
fig_corr = px.imshow(img=df_corr, text_auto=True, aspect="auto")
fig_corr.show()

Correlation analysis will return three possible scenarios: (1) positive correlation; (2) negative correlation; and (3) no correlation.

1. Positive correlation: when two variables that have a correlation grow or decrease together, in other words, they have a direct relationship;
2. Negative correlation: when two variables have a correlation but when one variable grows the other decreases, or vice versa;
3. No correlation: when the growth or decline of one variable has no effect on the other variable.

How can we interpret the values that ρ can take?

- ρ = 0.9 to 1 (positive or negative): very strong correlation;
- ρ = 0.7 to 0.9 (positive or negative): strong correlation;
- ρ = 0.5 to 0.7 (positive or negative): moderate correlation;
- ρ = 0.3 to 0.5 (positive or negative): weak correlation;
- ρ = 0 to 0.3 (positive or negative): no correlation.

So, we can see that there is a strong correlation or a strong dependency between each variable.

In [33]:
fig_hist_h_study = px.histogram(data_frame=df_h_study.HR_STUDY, text_auto=True)
fig_hist_h_study.show()

In [34]:
fig_hist_eval_score = px.histogram(data_frame=df_h_study.EVALUATION_SCORE, text_auto=True)
fig_hist_eval_score.show()

> ## Feature engineering
>

> ## Modeling
>

> ### Model 1
> Standard

> #### Building
>

In [35]:
X = df_h_study.HR_STUDY
y = df_h_study.EVALUATION_SCORE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [36]:
X_train = np.array(X_train).reshape(-1, 1)
X_test = np.array(X_test).reshape(-1, 1)
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

In [37]:
lr1 = LinearRegression()
lr1.fit(X=X_train, y=y_train)

LinearRegression()

In [38]:
lr1_pred = lr1.predict(X_test)

The coefficients

In [39]:
lr1.coef_

array([[1.4582253]], dtype=float32)

In [40]:
lr1.intercept_

array([33.47753], dtype=float32)

The mean squared error

In [41]:
mean_squared_error(y_test, lr1_pred)

214.1315296728518

In [42]:
mean_absolute_error(y_true=y_test, y_pred=lr1_pred)

11.564651233303932

> #### Evaluation
>

In [43]:
r2_score(y_test, lr1_pred)

0.43479554926280595

> ### Model 2
>

> #### Building
>

In [44]:
X = df_h_study.HR_STUDY
y = df_h_study.EVALUATION_SCORE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

To solve problem with shape.

In [45]:
X_train = np.array(X_train).reshape(-1, 1)
X_test = np.array(X_test).reshape(-1, 1)
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

Using simple linear regression from sklearn.

In [46]:
lr2 = LinearRegression()
lr2.fit(X=X_train, y=y_train)

LinearRegression()

In [47]:
lr2_pred = lr2.predict(X_test)

The coefficients

In [48]:
lr2.coef_

array([[1.46173]], dtype=float32)

In [49]:
lr2.intercept_

array([33.89179], dtype=float32)

The mean squared error

In [50]:
mean_squared_error(y_test, lr2_pred)

259.3418644797178

In [51]:
mean_absolute_error(y_true=y_test, y_pred=lr2_pred)

12.297504670678117

> #### Evaluation
>

The coefficient of determination: 1 is perfect prediction

In [52]:
r2_score(y_test, lr2_pred)

0.4405253893450902

#### Visualization

In [53]:
fig = px.scatter(x=X_test[:,0], y=y_test[:,0])
fig.show()

In [54]:
fig = px.line(x=X_test[:,0], y=lr2_pred[:,0])
fig.show()

> ### Model 3
> Stantard

> #### Building
>

> #### Evaluation
>

> ### Model 4
> Stantard

> #### Building
>

> #### Evaluation
>

> ## Model Deployment
>

In [55]:
lr1.predict([[18.06]])

array([[59.81307945]])

In [56]:
lr2.predict([[18.06]])

array([[60.29063234]])